In [26]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (150 kB/s)
Reading package lists... Done


In [27]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [28]:
# Create the vine_table DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [29]:
# Filter the data and create a new dataframe to retrieve all of the rows where the total_votes column is equal or greater than 20 
vine_df_20 = vine_df.filter(vine_df["total_votes"] >= "20")
vine_df_20.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [31]:
# Filter the new dataframe and create new dataframe to retrieve all rows where the number of helpful_votes/total votes is equal to or greater than 50% 
new_vine_df = vine_df_20.withColumn("vote_calc", vine_df_20["helpful_votes"]/ vine_df_20["total_votes"]).filter("vote_calc>=0.5")
new_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         vote_calc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|            0.9375|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|0.9807692307692307|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|0.8055555555555556|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|0.9866666666666667|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|0.8846153846153846|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|0.9251700680272109|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|0.9047619047619048|
|R1P0LEEJHH09L3|          5|  

In [50]:
# Filter the dataframe that retrieves all of the rows where a review was written as part of the vine program (paid) vine == "Y"
total_vine_reviews = new_vine_df.filter(new_vine_df["vine"] == "Y").count()
total_vine_reviews

33

In [51]:
# Filter the dataframe that retrieves all of the rows where a review was NOT written as part of the vine program (unpaid) vine == "N"
total_unpaid_reviews = new_vine_df.filter(new_vine_df["vine"] == "N").count()
total_unpaid_reviews

45388

In [34]:
# Determine the total number of reviews 
total_reviews = new_vine_df.count()
total_reviews

45421

In [ ]:
# Determine the total number of five star reviews 
total_five_star_reviews = new_vine_df.filter(new_vine_df["star_rating"] == 5).count()
total_five_star_reviews

23748

In [52]:
# Determine the percentage of 5 star reviews for paid reviews (vine = Y)
total_paid_five_star = new_vine_df.filter((new_vine_df["vine"] == "Y")&(new_vine_df["star_rating"] == 5)).count()
total_paid_five_star

percent_five_star_paid = total_paid_five_star/total_vine_reviews * 100
percent_five_star_paid

45.45454545454545

In [53]:
# Determine the percentage of 5 star reviews for unpaid reviews (vine = N)
total_unpaid_five_star = new_vine_df.filter((new_vine_df["vine"] == "N")&(new_vine_df["star_rating"] == 5)).count()
total_unpaid_five_star

percent_five_star_unpaid = total_unpaid_five_star/total_unpaid_reviews * 100
percent_five_star_unpaid

52.28915131752886

In [48]:
# Total number of unpaid (vine == N) 5 star reviews
total_unpaid_five_star

23733

In [49]:
# Total number of paid (vine == Y) 5 star reviews
total_paid_five_star

15